# **Step 4:** Detect Visual Alteration Model

This model use Dice coeff.
Separate the dataset into 3 repositories Train, Validate, and Testing with each having 2 repositories for the forged images and their respective masks.
Training repository having 70% of images
Validation repository having 15% of images
Testing repository having 15% of images

In [ ]:


def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = keras.backend.flatten(y_true)
    y_pred_f = keras.backend.flatten(y_pred)
    intersection = keras.backend.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (keras.backend.sum(y_true_f) + keras.backend.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = keras.losses.binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

def weighted_bce_dice_loss(y_true, y_pred, weight=0.5):
    """
    Calculate weighted binary cross-entropy and dice loss.

    Parameters:
    - y_true: True labels.
    - y_pred: Predictions.
    - weight: Weight for the positive class. The weight for the negative class will be 1.

    Returns:
    - Weighted loss value.
    """
    # Calculate weighted binary cross-entropy loss
    bce = keras.losses.binary_crossentropy(y_true, y_pred)
    bce_weighted = bce * (y_true * weight + (1 - y_true) * (1 - weight))

    # Calculate dice loss
    dice = dice_loss(y_true, y_pred)

    # Combine losses
    loss = bce_weighted + dice
    return loss




img_height = 512
img_width = 512


batch_size = 1

image_datagen_train = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,

)

mask_datagen_train = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,

)

image_generator_train = image_datagen_train.flow_from_directory(
    '/content/data/train/Forged',
    class_mode=None,
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)

mask_generator_train = mask_datagen_train.flow_from_directory(
    '/content/data/train/Mask',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    color_mode='grayscale',
    class_mode=None,
    seed=2)




image_datagen_valid = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    )
mask_datagen_valid = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
    )

image_generator_valid = image_datagen_valid.flow_from_directory(
    '/content/data/val/Forged',
    class_mode=None,
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)

mask_generator_valid = mask_datagen_valid.flow_from_directory(
    '/content/data/val/Mask',
    target_size=(img_height,img_width),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode=None,
    seed=2)

train_generator = zip(image_generator_train, mask_generator_train)
validation_generator = zip(image_generator_valid, mask_generator_valid)


model = keras.Sequential()


model.add(keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu', input_shape=(img_height, img_width, 3)))
model.add(keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(256, (3, 3), padding = 'same', activation = 'relu'))

model.add(keras.layers.UpSampling2D(size=(2, 2)))
model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(128, (3, 3), padding = 'same', activation = 'relu'))

model.add(keras.layers.UpSampling2D(size=(2, 2)))
model.add(keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(64, (3, 3), padding = 'same', activation = 'relu'))

model.add(keras.layers.UpSampling2D(size=(2, 2)))
model.add(keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))
model.add(keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = 'relu'))

model.add(keras.layers.Conv2D(1, (1, 1), padding = 'same', activation='sigmoid'))

model.summary()

model.compile(
    'Adam',
    loss=bce_dice_loss,
    metrics=[dice_coeff, 'accuracy'],
)

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='./logs/seg_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}_val_acc-{val_accuracy:.4f}.h5', #'./epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}_val_acc-{val_accuracy:.4f}.h5'
                                   monitor='val_loss',
                                   verbose=1,
                                   save_best_only=True,
                                   save_weights_only=False,
                                    mode='auto',
                                   period=1)

learning_rate_scheduler = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=0.0000001)

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

cal_b = [model_checkpoint]

nb_epochs = 20

history = model.fit_generator(
        train_generator,
        epochs=nb_epochs,
        validation_data=validation_generator,
        verbose=1,
        steps_per_epoch=30//batch_size,
        validation_steps=10//batch_size,
        callbacks=cal_b)




In [ ]:
plt.figure()
plt.plot(range(1, len(history.history['accuracy'])+1), history.history['accuracy'])
plt.plot(range(1, len(history.history['val_accuracy'])+1), history.history['val_accuracy'])
plt.ylim(0, 1)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.figure()
plt.plot(range(1, len(history.history['loss'])+1), history.history['loss'])
plt.plot(range(1, len(history.history['val_loss'])+1), history.history['val_loss'])
plt.ylim(0, 2)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

image_datagen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
image_generator_test = image_datagen_test.flow_from_directory(
    '/content/data/test/Forged',
    class_mode=None,
    color_mode='rgb',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)
mask_datagen_test = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
mask_generator_test = mask_datagen_test.flow_from_directory(
    '/content/data/test/Mask',
    class_mode=None,
    color_mode='grayscale',
    target_size=(img_height,img_width),
    batch_size=batch_size,
    seed=2)
# Evaluate the model
def combine_generator(gen1, gen2):
    while True:
        yield(gen1.next(), gen2.next())

test_generator = combine_generator(image_generator_test, mask_generator_test)
steps = len(image_generator_test)
loss,dice_coeff,accuracy = model.evaluate_generator(test_generator, steps=steps)

print(f'Test loss: {loss:.4f}')
print(f'Test dice_coeff: {dice_coeff:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

Test loss: 0.4019

Test iou_score: 0.8074

Test dice_coeff: 0.8834

Test accuracy: 0.4943